In [22]:
from github import Github
from github import Auth
import os
from dotenv import load_dotenv

load_dotenv()
auth = Auth.Token(os.environ['GITHUB_TOKEN'])
g = Github(auth=auth)

repo = g.get_repo("ktrnka/update-your-readme")

def repo_contents_to_markdown(repo):
    markdown = ""
    for content in repo.get_contents(""):
        markdown += f"{content.path}\n"
    return markdown

print(repo_contents_to_markdown(repo))

.gitignore
Pipfile
Pipfile.lock
README.md



In [21]:

def pull_request_to_markdown(pr):
    text = f"""
# [{pr.title}]){pr.html_url}
{pr.body or 'No description provided.'}

"""
    for file in pr.get_files():
        patch = file.patch or "Can't render patch."
        text += f"""## {file.filename}\n{patch}\n\n"""

    return text

pulls = repo.get_pulls(state='open')
for pr in pulls:
    print(pull_request_to_markdown(pr))



# [Test PR])https://github.com/ktrnka/update-your-readme/pull/1
No description provided.

## Pipfile
@@ -7,6 +7,8 @@ name = "pypi"
 PyGithub = "*"
 requests = "*"
 python-dotenv = "*"
+langchain = "*"
+langchain-anthropic = "*"
 
 [dev-packages]
 pytest = "*"

## src/test_github.ipynb
Can't render patch.




In [15]:
readme = repo.get_contents("README.md")
print(readme.decoded_content.decode())

In [25]:
from pydantic import BaseModel
from typing import Optional
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import PromptTemplate


model = ChatAnthropic(model='claude-3-5-sonnet-20240620')



class UpdateRecommendation(BaseModel):
    should_update: bool
    reason: Optional[str]
    patch: Optional[str]


# Generate the LangChain Claude prompt
prompt = PromptTemplate.from_template("""
# README Content
{readme_content}

# File Tree
{file_tree}

# PR Patch
{pr_patch}

# Task
Based on the above information, please provide a structured output indicating:
A) Should the README be updated?
B) Why?
C) A patch of the update to make.
""")

pipeline = prompt | model.with_structured_output(UpdateRecommendation)
pr = repo.get_pull(1)
result = pipeline.invoke({
    "readme_content": repo.get_contents("README.md").decoded_content.decode(),
    "file_tree": repo_contents_to_markdown(repo),
    "pr_patch": pull_request_to_markdown(pr)
})
result

UpdateRecommendation(should_update=True, reason="The pull request includes updates to the Pipfile to add new dependencies (langchain and langchain-anthropic) which are not currently reflected in the README file. It's a good practice to keep the README up-to-date with the project's dependencies and other relevant information.", patch='```markdown\n# Dependencies\n\nThis project uses the following dependencies:\n\n- PyGithub\n- requests \n- python-dotenv\n- langchain\n- langchain-anthropic\n```')

In [26]:
print(f"""
Should update: {result.should_update}

Reason:
{result.reason}

Patch:
{result.patch}
""")


Should update: True

Reason:
The pull request includes updates to the Pipfile to add new dependencies (langchain and langchain-anthropic) which are not currently reflected in the README file. It's a good practice to keep the README up-to-date with the project's dependencies and other relevant information.

Patch:
```markdown
# Dependencies

This project uses the following dependencies:

- PyGithub
- requests 
- python-dotenv
- langchain
- langchain-anthropic
```

